# Tutorial 3: Learning

In [ ]:
import libspn as spn
import tensorflow as tf

### Building a Test Graph with Random Weights

In [ ]:
iv_x = spn.IVs(num_vars=2, num_vals=2, name="iv_x")
sum_11 = spn.Sum((iv_x, [0,1]), name="sum_11")
sum_12 = spn.Sum((iv_x, [0,1]), name="sum_12")
sum_21 = spn.Sum((iv_x, [2,3]), name="sum_21")
sum_22 = spn.Sum((iv_x, [2,3]), name="sum_22")
prod_1 = spn.Product(sum_11, sum_21, name="prod_1")
prod_2 = spn.Product(sum_11, sum_22, name="prod_2")
prod_3 = spn.Product(sum_12, sum_22, name="prod_3")
root = spn.Sum(prod_1, prod_2, prod_3, name="root")
iv_y = root.generate_ivs(name="iv_y")
spn.generate_weights(root, initializer=tf.initializers.random_uniform(0, 1))

### Visualizing the SPN Graph

In [ ]:
spn.display_spn_graph(root)

### Specify Training Data

In [ ]:
iv_x_arr=[[0,0],[0,0],[1,1],[1,1],[1,1],[0,1],[0,1],[0,1]]
iv_y_arr=[[-1]] * len(iv_x_arr)

### Add Learning Ops

In [ ]:
init_weights = spn.initialize_weights(root)
learning = spn.EMLearning(root, initial_accum_value=2)
init_learning = learning.reset_accumulators()
accumulate_updates = learning.accumulate_updates()
update_spn = learning.update_spn()
likelihood = tf.reduce_mean(learning.value.values[root])

### Run Learning

In [ ]:
epoch = 0
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    while run():
        likelihood_arr, _ = sess.run([likelihood, accumulate_updates],
                                  feed_dict={iv_x:iv_x_arr, iv_y:iv_y_arr})
        print("Avg. Likelihood: %s" % (likelihood_arr))
        sess.run(update_spn)
        epoch+=1
        if epoch > 10:
            break